# ESTRUTURAS DE DADOS

## CDIA20P2 | Semana 06

## Questão Dirigida

Como fazer o agente percorrer todo o labirinto?

## Parte 1 - Evolução do Projeto

### A classe ```Waze```

A classe ```Waze```, que fica no arquivo ```percursos.py``` é a classe responsável por calcular percursos do nosso agente. Para calcular os percursos, a classe ```Waze``` deverá ter informações do labirinto, conter uma lista e os algoritmos de busca. 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from celula import Celula

class Waze:

    def __init__(self, labirinto):
        """ Construtor. Inicializa o objeto com uma referência ao labirinto,
            uma lista de coordenadas a ser seguida pelo agente, e um atributo
            que indica se o percurso já foi finalizado.
        """
        self.labirinto = labirinto
        self.lst_coord = []
        self.finalizado = None

    def obter_prox_coord(self):
        """ Retorna a próxima coordenada ao agente """
        if (len(self.lst_coord) > 0):
            item = self.lst_coord.pop(0)
            # Caso a lista fique vazia após a retirada (pop) do elemento,
            # significa que o percurso acabou
            if (len(self.lst_coord) == 0):
                self.finalizado = True
            return item
        return

    def fim_percurso(self):
        """ Mostra se o percurso está finalizado (True ou False) """
        return self.finalizado == True

    def esta_sem_coord(self):
        """ Mostra se a lista de coordenadas está vazia (True ou False) """
        return len(self.lst_coord) == 0

    def gerar_percurso(self, celula):
        """ Gera um percurso no labirinto a partir de uma determinada célula """
        lab = self.labirinto # Para melhorar a legibilidade
        visitadas = [] # Indica quais células já foram visitadas
        # lst_coord: indica as coordenadas que o agente deve seguir para
        #            percorrer todo o labirinto
        lst_coord = []

        # Algoritmo de busca em profundidade
        self.__dfs(celula, visitadas, lst_coord)

        self.lst_coord = lst_coord

        # Logo após ter gerado o percurso, o agente ainda não o percorreu.
        # Por esta razão, este atributo é inicializado como False
        self.finalizado = False

    def __dfs(self, celula, visitadas, lst_coord):
        """ Implementação do algoritmo de busca em profundidade """
        if (celula in visitadas):
            return
        lst_coord.append(celula)
        visitadas.append(celula)

        lab = self.labirinto # Para melhorar a legibilidade
        vizinhas = lab.obter_vizinhos(celula)
        for cel_vizinha in vizinhas:
            if (cel_vizinha not in visitadas):
                self.__dfs(cel_vizinha, visitadas, lst_coord)
                lst_coord.append(celula)

### A Classe ```Agente``` 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from turtle import *

# add as seguintes bibliotecas
from percursos import Waze

class Agente:

    # inclui cor e tartaruta pro agente
    def __init__(self, id, tam_agente, cor=None):
        # add uma identificação única pro agente
        self._id = id
        self._tam_agente = tam_agente

        # add uma tartaruga específica pro agente
        self._turtle = Turtle()
        self._turtle.hideturtle()

        # define a cor do agente
        self._cor = cor

        self._waze = None

    # adiciona um labirinto
    def add_labirinto(self, lab):
        self._labirinto = lab
        self._posicao = lab.cel_aleatoria()

    # Note que o nome do método mudou um pouco
    def desenhar_se(self, posicao=None):
        """ Leva a tartaruga até a posição (x,y) e desenha por exemplo um círculo
            para representar o agente (i.e., pacman, fantasmas)
        """
        self._turtle.clear()
        if (not posicao):
            posicao = self._posicao

        x, y = posicao.coord_turt_centralizada()
        self._turtle.up()
        self._turtle.goto(x , y)
        self._turtle.down()
        self._turtle.dot(self._tam_agente, self._cor)

    """ Métodos de percurso """

    def add_percurso(self):
        if (not self._waze):
            self._waze = Waze(self._labirinto)

    def percorrer(self):
        """ Percorrer significa seguir passar por todas as celulas do labirinto """
        pos_agente = self._posicao # para melhorar a legibilidade do cód

        self.add_percurso()
        if (self._waze.fim_percurso()):
            self._waze = None
            return True

        if (self._waze.esta_sem_coord()):
            self._waze.gerar_percurso(pos_agente)

        if (not self._waze.esta_sem_coord()):
            self._posicao = self._waze.obter_prox_coord()
            self.desenhar_se()

        return False

### A classe ```Labirinto```

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from utils import floor
from turtle import *
import numpy as np

# Acrescenta esta biblioteca ao labirinto
from celula import Celula

class Labirinto:

    def __init__(self, dim, tam_celula):
        """ Construtor do Labirinto """
        self._matriz = self.ler_matriz_fixa() # Carrega uma matriz fixa
        self._dim = dim # Atributo que armazena a dimensão da matriz
        self._tam_celula = tam_celula # Atributo que armazena o tamanho da célula

        self._turtle = Turtle() # A tartaruga que desenha o caminho do labirinto
        self._turtle.hideturtle() # Esconde a tartaruga

    def criar_celula(self, coord_matr=None, coord_turt=None):
        """ Cria uma célula com o tamanho de célula e dimensão de _matriz
            definidas como atributos na classe Labirinto
        """
        return Celula(coord_matr, coord_turt, self._tam_celula, self._dim)

    def ler_matriz_aleatoria(self, dim):
        """ Retorna uma matriz quadrada na dimensão especificada com números
        aleatórios (0's e 1's)
        """
        return np.random.randint(2, size=(dim,dim))

    def ler_matriz_fixa(self):
        """ Retorna uma matriz fixa """

        return  [[1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,1,1,1,1,1],
                 [0,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0],
                 [0,0,1,1,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,0],
                 [0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,0],
                 [0,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,0,0],
                 [1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,1,1],
                 [1,0,1,0,0,0,0,0,1,0,1,1,1,1,1,0,1,0,1,0],
                 [1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0],
                 [1,0,0,0,1,1,0,1,0,0,1,0,1,1,0,0,1,1,1,1],
                 [1,1,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,1,1],
                 [1,0,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,1,1,0],
                 [1,0,1,1,1,0,0,0,0,1,1,1,0,0,1,1,0,1,1,0],
                 [0,1,0,1,1,1,0,0,0,1,1,0,0,1,1,0,1,0,1,0],
                 [0,1,1,0,0,1,0,1,1,1,0,0,1,1,1,1,1,1,1,0],
                 [0,0,1,1,1,1,1,1,0,1,0,1,1,0,0,0,1,1,1,1],
                 [1,1,1,0,1,1,0,1,0,0,1,1,1,0,0,1,0,1,0,1],
                 [1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,1,0,0],
                 [1,0,1,1,0,0,1,1,0,1,0,1,0,1,1,0,0,1,0,0],
                 [1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1],
                 [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]]

    def criar_labirinto(self, p1=420, p2=420, p3=370, p4=0):
        """ Cria o gráfico do labirinto baseado nos valores da matriz """
        setup(p1, p2, p3, p4)

        # Para cada linha da matriz
        for lin in range(self._dim):
            # Para cada coluna da matriz
            for col in range(self._dim):
                # Testa se a coordenada da matriz (lin, col) é caminho (=1)
                if (self._matriz[lin][col] == 1):
                    # Em caso positivo, transforma em coordenada Turtle.
                    # Atenção: Numa coordenada Turtle (x,y), o eixo x refere-se à coluna e o eixo y à linha
                    # Numa coordenada da matriz (lin, col), o primeiro elemento é a linha e o segundo a coluna

                    celula = self.criar_celula(coord_matr=(lin, col))
                    # Pinta a celula na posição (x,y) com a cor especificada
                    self.desenhar_celula(celula, 'blue')

                    self.desenhar_pastilha(celula, 'white')

    def cel_aleatoria(self):
        """ Retorna os índices de uma posição que seja caminho
        """
        i, j = np.random.randint(self._dim, size=(2))
        while (not self.eh_caminho(i, j)):
            i, j = np.random.randint(self._dim, size=(2))

        celula = self.criar_celula(coord_matr=(i,j))
        return celula

    def desenhar_pastilha(self, celula, cor):
        """ Leva a tartaruga até a posição (x,y) e desenha por exemplo um círculo
            para representar a pastilha
        """
        x, y = celula.coord_turt_centralizada()
        self._turtle.up()
        self._turtle.goto(x, y)
        self._turtle.down()
        self._turtle.dot(3, cor)

    def eh_caminho(self, lin, col):
        """ Dada uma matriz quadrada, retorna True quando (lin, col) == 1 e
            False caso contrário.
            Por exemplo, na matriz a seguir:
            [[ 1  0  0 ]
             [ 0  1  0 ]
             [ 0  0  1 ]]
            a chamada de função 'eh_caminho(0,0)' retorna True e
            a chamada de função 'eh_caminho(0,1)' retorna False
        """
        return lin >= 0 and col >= 0 and                    \
               lin < self._dim and col < self._dim and      \
               self._matriz[lin][col] == 1

    def desenhar_celula(self, celula, cor):
        """ Dada uma coordenada (x, y) do Turtle, desenha um quadrado (célula) na posição """
        x, y = celula.coord_turtle()
        self._turtle.color(cor)
        self._turtle.up()
        self._turtle.goto(x,y)
        self._turtle.down()
        self._turtle.begin_fill()
        for _ in range(4):
            self._turtle.forward(self._tam_celula)
            self._turtle.left(90)
        self._turtle.end_fill()
        self._turtle.up()

    def obter_vizinhos(self, celula):
        """ Retorna os vizinhos de uma celula """
        lin, col = celula.coord_matriz()
        vizinhos = []

        if (self.eh_caminho(lin-1, col)):
            vz_cima = self.criar_celula(coord_matr=(lin-1, col))
            vizinhos.append(vz_cima)

        if (self.eh_caminho(lin, col-1)):
            vz_esquerdo = self.criar_celula(coord_matr=(lin, col-1))
            vizinhos.append(vz_esquerdo)

        if (self.eh_caminho(lin, col+1)):
            vz_direito = self.criar_celula(coord_matr=(lin, col+1))
            vizinhos.append(vz_direito)

        if (self.eh_caminho(lin+1, col)):
            vz_baixo = self.criar_celula(coord_matr=(lin+1, col))
            vizinhos.append(vz_baixo)

        return vizinhos

### A classe ```Celula```

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from utils import floor

class Celula:
    """
    O propósito desta classe é converter as coordenadas da matriz em coordenadas
    Turtle e vice-versa. Um objeto Celula criado a partir das coordenadas da matriz
    irá convertê-las na criação (i.e., no construtor) em coordenadas Turtle e vice-versa.
    """

    # Uma coordenada é apenas uma tupla
    def __init__(self, coord_matr=None, coord_turt=None, tam_cel=None, dim=None):
        self._tam_celula = tam_cel
        self._dim = dim

        if (not coord_turt == None):
            # Se passar uma coordenada Turtle, converte pra matriz
            self._coord_turt = coord_turt
            self._coord_matr = self.em_coord_matriz(coord_turt)

        if (not coord_matr == None):
            # Se passar uma coordenada da matriz, converte pra Turtle
            self._coord_matr = coord_matr
            self._coord_turt = self.em_coord_turtle(coord_matr)

    def em_coord_turtle(self, coord_matr):
        """ Dados os índices da matriz (lin, col), retorna as coordenadas do Turtle correspondentes.
            Por exemplo, numa matriz quadrada de dimensão 20, com tamanho de célula 20,
            a chamada de função 'em_coord_turtle(0,0)' deve retornar (-200,200) e a
            chamada de função 'em_coord_turtle(10,10)' deve retornar (0,0)
        """
        lin, col = coord_matr
        meio = self._dim // 2
        x = (col - meio) * self._tam_celula
        y = (meio - lin) * self._tam_celula
        return x, y

    def em_coord_matriz(self, coord_turt):
        """ Dada uma coordenada do Turtle (x,y), retorna os índices correspondentes da matriz
            Por exemplo, numa matriz quadrada de dimensão 20, com tamanho de célula 20,
            a chamada de função 'em_coord_matriz(-200, 200)' deve retornar (0,0) e a
            chamada de função 'em_coord_matriz(0, 0)' deve retornar (10,10).
        """
        x, y = self.chao_da_celula( coord_turt )
        meio = self._dim // 2
        lin = int(meio - (y / self._tam_celula))
        col = int(meio + (x / self._tam_celula))
        return lin, col

    def chao_da_celula(self, coord_turt):
        """ Dadas coordenadas do Turtle (x,y), retorna as coordenadas do início de uma célula.
            Por exemplo, na celula da origem com tamanho 20, a coordenada Turtle (10,10)
            representa o meio da célula. A chamada de função 'chao_da_celula(10, 10)' retorna
            as coordenadas de início dessa célula (0,0
        """
        x, y = coord_turt
        chao_x = int(floor(x, self._tam_celula))
        chao_y = int(floor(y, self._tam_celula))
        return chao_x, chao_y

    def coord_turt_centralizada(self):
        """ Retorna uma coordenada Turtle centralizada na célula """
        x, y = self.coord_turtle()
        x += self._tam_celula // 2
        y += self._tam_celula // 2
        return x, y

    def coord_matriz(self):
        """ Retorna a coordenada de matriz """
        return self._coord_matr

    def coord_turtle(self):
        """ Retorna a coordenada do Turtle """
        return self._coord_turt

    def __eq__(self, other):
        """ Compara este (self) objeto com outro (other) considerando somente
            as coordenadas da matriz
        """
        return self._coord_matr == other._coord_matr

### ```pacman.py```

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from agente import Agente
from labirinto import Labirinto
from turtle import *
from time import sleep

def main():
    tracer(False)
    hideturtle()
    bgcolor('black')

    dimensao_da_matriz = 20
    tam_celula = 20

    # Cria o labirinto
    lab = Labirinto(dimensao_da_matriz, tam_celula)
    lab.criar_labirinto()

    # Cria o agente
    tam_agente = 20
    agente = Agente(0, tam_agente, "yellow")
    agente.add_labirinto(lab)

    terminou_percurso = False
    intervalo_entre_frames = 0.1
    while (not terminou_percurso):
        terminou_percurso = agente.percorrer()
        # Atualiza o turtle e finaliza
        update()
        sleep(intervalo_entre_frames)

    done()

main()

### Parte 2 - Semana que vem

## Tarefa para a próxima aula (08/09)

Elabore um relatório que explique a simulação do Pac-Man (no estado atual) para uma pessoa leiga que não tenha familiaridade com o que estamos trabalhando. **Não expliquem o algoritmo DFS ainda.**

Sugestões de perguntas a serem respondidas para guiar a elaboração do relatório.

1. Quais são as classes da aplicação? Para que serve cada classe?

1. Em linhas gerais, o que faz a função ```main()``` no arquivo ```pacman.py```?

1. Por que a classe ```Labirinto``` possui os seguintes atributos: ```_matriz```, ```_dim``` e ```_tam_celula```?

1. Explique a lógica do construtor da classe ```Celula```.

1. Que métodos da classe ```Celula``` são chamados na classe ```Labirinto```?

1. Por que foi necessário incluir o método ```criar_celula()``` na classe ```Labirinto```?

1. O que representa o atributo ```_posicao``` na classe ```Agente```?

1. Qual o papel do método ```add_labirinto()``` na classe ```Agente```? O que acontece quando esse método é chamado na função ```main()```?

1. A tartaruga utilizada para desenhar o caminho e a pastilha na classe ```Labirinto``` é a mesma utilizada pelo ```agente```? Considere o caso de serem criados dois agentes. Eles utilizarão a mesma tartaruga?

1. Qual a sequência de chamadas de métodos a partir do momento em que ocorre a chamada de método ```agente.percorrer()``` na função ```main()``` do arquivo ```pacman.py```? *Não é necessário mostrar repetições de chamada de método*.

1. Por que é necessário envolver a chamada de método ```agente.percorrer()``` em uma estrutura de repetição ```while```?